In [1]:
#import proj_paths
import edens as e
#import analytical_funcs as a

# general python libraries
import time, importlib, sys
from tqdm import tqdm_notebook as tqdm
import pylab as pl, numpy as np, glob, pdb, scipy, scipy.stats
from numpy import log10 as log
import h5py
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import yt
import unyt as u
from unyt import kpc, km, cm, s, Msun, gram, dimensionless
try: # because it can only be defined once
    u.define_unit("Msun10", Msun*1e10)
except:
    pass

# FIRE studio libraries
sys.path.append('/Users/jonathanstern/Dropbox/other_repositories/')
sys.path.append('/Users/jonathanstern/Dropbox/other_repositories/FIRE_studio/')
import abg_python
import firestudio
from firestudio.studios.gas_studio import GasStudio
#from firestudio.studios.star_studio import StarStudio

palettable colormaps are not installed
don't have phil's colormaps


# Align data to cold gass

In [2]:
directory = '/Users/jonathanstern/Dropbox/github_repositories/gizmo_analysis/ipynb/subsonic_solution/snapshots/good/'
MW_simfile = directory+"snapshot_071.hdf5"
NGC891_simfile = directory+"snapshot_049.hdf5"


In [3]:
snap_path_write = directory
if True:
    # load snapshot to dictionary
    gasdata = e.load_data(snap_path_write, partType=["gas"], keys=["Coordinates","Velocities","ElectronAbundance","InternalEnergy","Metallicity"])
    starsdata = e.load_data(snap_path_write, partType=["stars"], keys=["Coordinates","Velocities"])

    r_gas = gasdata["gas"]["Coordinates"] * kpc
    v_gas = gasdata["gas"]["Velocities"] * km/s
    r_stars = starsdata["stars"]["Coordinates"] * kpc
    v_stars = starsdata["stars"]["Velocities"] * km/s

    # calc temeperature from data
    helium_fraction = gasdata["gas"]["Metallicity"][:,1] * dimensionless
    int_energy = gasdata["gas"]["InternalEnergy"] * (km/s)**2
    e_a = gasdata["gas"]["ElectronAbundance"] * dimensionless
    t_gas = e.temperature(int_energy, e_a, 5/3, helium_fraction)

    # choose only main halo gas (within 50kpc radius around center)
    gas_main_halo_index = r_gas[:,0]**2 + r_gas[:,1]**2 + r_gas[:,2]**2 <50**2
    r_gas_main_halo = r_gas[gas_main_halo_index]
    v_gas_main_halo = r_gas[gas_main_halo_index]
    
    # choose only cold ( < 5e4K ) gas within main halo
    t_gas_main_halo = t_gas[gas_main_halo_index]
    r_cold_gas = r_gas_main_halo[t_gas_main_halo < 5000 * u.K]
    v_cold_gas = v_gas_main_halo[t_gas_main_halo < 5000 * u.K]
    
    # calculate angular momentum of main halo cold gas only (cgm)
    L_cold = e.calc_angular_momentum(r_cold_gas, v_cold_gas)[2]
    
    # align all coordinates
    r_stars_aligned = (e.rotate_coords(r_stars, L_cold))
    v_stars_aligned = (e.rotate_coords(v_stars, L_cold))
    r_gas_aligned = (e.rotate_coords(r_gas, L_cold))
    v_gas_aligned = (e.rotate_coords(v_gas, L_cold))
    
    # write data to file
    e.write_to_fire_sim( snap_path_write, r_gas_aligned, v_gas_aligned, r_stars_aligned, v_stars_aligned)

/Users/jonathanstern/Dropbox/github_repositories/gizmo_analysis/ipynb/subsonic_solution/edens.py:63: RuntimeWarning: invalid value encountered in float_scalars
  theta = np.arccos(n_vec[2]/R)


# parameters

In [4]:
h=0.702
r_max=50
z_width=50
filedir=directory
projection_output_filename=directory

In [5]:
axes = {0: [1,0,0], 1: [0,1,0], 2: [0,0,1]}
axes_names = {0:"x",1:"y",2:"z"}
print(list(axes.items()))

[(0, [1, 0, 0]), (1, [0, 1, 0]), (2, [0, 0, 1])]


# plot a projection function

In [6]:
def plot(map, simsnapname, los_axis):

    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2, top=2)
    
    ksz_Norm = matplotlib.colors.Normalize(vmax=0.1,vmin=-0.1)
    im = ax.imshow(map * u.Tcmb.v * 1e6,cmap='seismic_r',extent=[-r_max, r_max, -r_max, r_max], norm = ksz_Norm)
    
    ax.set_xlabel(("x" if los_axis else "y") + "[kpc]", size=14)
    ax.set_ylabel("z [kpc]", size=14)
    ax.tick_params(length=6, width=2, labelsize=13)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    ksz_cb = fig.colorbar(im, cax=cax, orientation='vertical')
    ksz_cb.ax.tick_params(labelsize=13)
    ksz_cb.set_label(label="kSZ [$\mu$K]", size=16)
    ax.tick_params(length=6, width=2, labelsize=13)

    #plt.savefig(os.path.join(proj_paths.PROJDIR,"firestudio_results/plots/", simsnapname + "_los_"+axes_names[los_axis]+".png"), bbox_inches="tight")

# make projections for all snapshots

In [7]:
#for simsnap in proj_paths.ALLSNAPS_WRITE:
if True:    
    #simname = (simsnap.split("/")[-2]).split("_")[0]
    #snap = int((simsnap.split("/")[-1]).split("_")[-1])
    #resolution = int((simsnap.split("/")[-2]).split("res")[-1])
    #simsnapname = simname + "_" + str(snap)

    # create the data dictionary
    datadic = e.load_data(directory, partType=["gas"], keys=["Coordinates","Velocities","Masses","Density","ElectronAbundance","SmoothingLength"])

    # rotate coordinates such that the los is on the z axis
    datadic_r = datadic
    for (los_axis, rot_vec) in list(axes.items())[:2]:
        rotated_r = e.rotate_coords(datadic["gas"]["Coordinates"], rot_vec)
        rotated_v = e.rotate_coords(datadic["gas"]["Velocities"], rot_vec)
        datadic_r["gas"]["Coordinates"] = rotated_r
        datadic_r["gas"]["Velocities"] = rotated_v
    
        # create sz field
        v_los = datadic_r['gas']['Velocities'][:,2] * km/s
        rho = datadic_r['gas']['Density'] * u.Msun10 / kpc**3
        e_a = datadic_r['gas']['ElectronAbundance'] * dimensionless
        n_e = 0.7 * rho * e_a / u.proton_mass
        sz = (v_los * n_e * u.sigma_thompson / u.c)
        datadic_r['gas']['SZ']= sz.value / 1e-3 * 3e18
        datadic_r['gas']['BoxSize']=60000
    
        # create projection 
        mystudio = GasStudio(
         snapnum = 49,
         snapdict = datadic_r['gas'],
         frame_half_width = r_max,
         frame_depth = z_width,
         quantity_name = 'SZ',
         take_log_of_quantity = False, 
         galaxy_extractor = False,
         pixels=1200,
         overwrite = True,
         savefig=False,      
         use_hsml=True,
         snapdir = None, 
         datadir= filedir
         )
        _, map = mystudio.projectImage([])

        if los_axis==1:
            map = map.T
        
        #np.save(os.path.join(proj_paths.PROJDIR,"firestudio_results/data/", simsnapname + "_los_"+axes_names[los_axis]+".npy"), map)
        
        plot(map,'NGC891',los_axis)

extra kwargs:
 ['galaxy_extractor']
Drawing None:49 to:/Users/jonathanstern/Dropbox/github_repositories/gizmo_analysis/ipynb/subsonic_solution/snapshots/good/
extracting cube
-done
Using provided smoothing lengths
------------------------------------------


OSError: dlopen(/Users/jonathanstern/Dropbox/other_repositories/FIRE_studio/firestudio/utils/gas_utils/HsmlAndProject_cubicSpline/HsmlAndProject.so, 0x0006): tried: '/Users/jonathanstern/Dropbox/other_repositories/FIRE_studio/firestudio/utils/gas_utils/HsmlAndProject_cubicSpline/HsmlAndProject.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/jonathanstern/Dropbox/other_repositories/FIRE_studio/firestudio/utils/gas_utils/HsmlAndProject_cubicSpline/HsmlAndProject.so' (no such file), '/Users/jonathanstern/Dropbox/other_repositories/FIRE_studio/firestudio/utils/gas_utils/HsmlAndProject_cubicSpline/HsmlAndProject.so' (not a mach-o file)

In [8]:
pdb.pm()

> /Users/jonathanstern/opt/anaconda3/lib/python3.9/ctypes/__init__.py(382)__init__()
    380 
    381         if handle is None:
--> 382             self._handle = _dlopen(self._name, mode)
    383         else:
    384             self._handle = handle

ipdb> p self._name
'/Users/jonathanstern/Dropbox/other_repositories/FIRE_studio/firestudio/utils/gas_utils/HsmlAndProject_cubicSpline/HsmlAndProject.so'
ipdb> q
